# Vergleich von Baseline-Modellen mit DeepSynergy

In diesem Notebook evaluiere ich klassische Regressionsmodelle (Random Forest, Ridge Regression) und vergleiche sie mit DeepSynergy. Alle Hyperparameter wurden aus dem Paper genommen.


## Setup: Importieren der Bibliotheken

Hier importiere ich alle benötigten Bibliotheken:
- `sklearn` für Modelltraining und Cross Validation
- `pickle` und `numpy` für Datenverarbeitung
- `matplotlib` für spätere Visualisierung


In [3]:
import numpy as np
import pickle
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


Matplotlib is building the font cache; this may take a moment.


## Random Forest mit Nested Cross Validation

Hier wird ein Random Forest Modell mithilfe von Cross Validation evaluiert.  
Die äußere Schleife nutzt 5 Folds für die Modellbewertung, die innere GridSearchCV nutzt 3 Folds zur Auswahl der besten Hyperparameter pro Fold.


In [6]:
# Load normalized dataset from pickle file
with open("data/data_test_fold0_tanh.p", "rb") as f:
    data = pickle.load(f)

# Training data for cross-validation (nested CV)
X_train_all = data[2]
y_train_all = data[6]

# Test data used for DeepSynergy evaluation
X_test = data[3]
y_test = data[7]

print("Training data:", X_train_all.shape, y_train_all.shape)
print("Test data:", X_test.shape, y_test.shape)


Training data: (36996, 8846) (36996,)
Test data: (9108, 8846) (9108,)


## Daten laden

Die Daten wurden aus einer vorbereiteten Pickle-Datei geladen, die bereits die tanh-Normalisierung enthält (wie im Paper verwendet).  
Für die Baseline-Vergleiche wurden `X_train_all` und `y_train_all` als vollständiger Trainingsdatensatz für die 5-fache Nested Cross Validation genutzt.  
Die Daten `X_test` und `y_test` dienen später zur separaten Evaluation des DeepSynergy-Modells.


## Random Forest mit Nested Cross Validation

Hier wird ein Random Forest Regressor mit einer 5-fachen äußeren und einer 3-fachen inneren Cross Validation evaluiert.  
Die Hyperparameter stammen aus dem Paper.  Getestet wurden 3 Varianten der Anzahl von Entscheidungsbäumen n_estimators.  
Mit GridSearchCV wird innerhalb jedes Outer-Folds der beste Wert für n_estimators gesucht.  
Die Bewertung erfolgt über den mittleren quadratischen Fehler (Mean Squared Error, MSE).


In [ ]:
# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [128, 512, 1024],
    'max_features': ['sqrt']
}

# Define inner and outer cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=1)

rf_mse_scores = []

for train_idx, test_idx in outer_cv.split(X_train_all):
    X_train, X_val = X_train_all[train_idx], X_train_all[test_idx]
    y_train, y_val = y_train_all[train_idx], y_train_all[test_idx]

    rf_model = RandomForestRegressor(random_state=0)
    grid = GridSearchCV(
        rf_model,
        param_grid_rf,
        cv=inner_cv,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    grid.fit(X_train, y_train)
    best_rf = grid.best_estimator_
    predictions = best_rf.predict(X_val)
    mse = mean_squared_error(y_val, predictions)
    rf_mse_scores.append(mse)

    print(f"Fold MSE: {mse:.2f} | Best Params: {grid.best_params_}")

# Final summary
print("\nRandom Forest MSEs:", rf_mse_scores)
print("Mean Random Forest MSE:", np.mean(rf_mse_scores))

### Ergebnisse (aus Terminallauf):

- Fold 1: 241.42
- Fold 2: 295.49
- Fold 3: 294.35
- Fold 4: 290.07
- Fold 5: 263.79

** MSE: 277.02**

Diese Ergebnisse zeigen, dass der Random Forest über alle Folds hinweg relativ stabile Vorhersageleistungen erzielt.  
Die Schwankungen zwischen den Folds sind nicht hoch, was für ein robustes Modell spricht.  
Mit einem durchschnittlichen MSE von 277.02 liefert Random Forest im Vergleich zu Ridge Regression und DeepSynergy eine sehr starke Baseline.


## Ridge Regression mit Nested Cross Validation

Auch Ridge Regression wird mit einer 5-fachen verschachtelten Cross Validation (Nested CV) evaluiert.   
Die getesteten Werte 0.1, 1.0, 10.0, 100.0 stammen ebenfalls aus dem DeepSynergy-Paper.  


In [ ]:
# Define hyperparameter grid for Ridge Regression
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0, 100.0]
}

ridge_mse_scores = []

# Loop through outer folds
for train_idx, test_idx in outer_cv.split(X_train_all):
    X_train, X_val = X_train_all[train_idx], X_train_all[test_idx]
    y_train, y_val = y_train_all[train_idx], y_train_all[test_idx]

    ridge = Ridge()
    grid_ridge = GridSearchCV(
        ridge,
        param_grid_ridge,
        cv=inner_cv,
        scoring='neg_mean_squared_error',
        n_jobs=-1  # use all CPU cores
    )
    grid_ridge.fit(X_train, y_train)
    best_ridge = grid_ridge.best_estimator_
    predictions = best_ridge.predict(X_val)
    mse = mean_squared_error(y_val, predictions)
    ridge_mse_scores.append(mse)

    print(f"Fold MSE: {mse:.2f} | Best alpha: {grid_ridge.best_params_['alpha']}")

# Final summary
print("\nRidge Regression MSEs:", ridge_mse_scores)
print("Mean Ridge Regression MSE:", np.mean(ridge_mse_scores))


### Ergebnisse (aus Terminallauf):

- Fold 1: 385.14
- Fold 2: 453.62
- Fold 3: 444.98
- Fold 4: 440.76
- Fold 5: 405.29

**MSE: 425.96**

Diese Ergebnisse zeigen deutlich, dass Ridge Regression im Vergleich zu Random Forest deutlich schlechter abschneidet.  
Die Fehlerstreuung ist relativ hoch und die Performance bleibt in allen Folds klar hinter dem Random Forest zurück.  
Das bestätigt, dass lineare Modelle für dieses Problem weniger geeignet sind.


## DeepSynergy (PyTorch-Reimplementierung)


Die Evaluation erfolgt nur auf dem festen Testset (`X_test`, `y_test`), da keine Cross Validation verwendet wurde.  
Daher ist der Vergleich zu den Baselines nur eingeschränkt direkt vergleichbar. Ich habe das Modell nichtmehr mit Cross Validation trainieren können, ich hoffe das hat Michael gemacht als er DeepSynergy nachgebaut hat? 


In [ ]:
# Define DeepSynergy model architecture
class DeepSynergyModel(nn.Module):
    def __init__(self, input_size):
        super(DeepSynergyModel, self).__init__()
        self.model = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(input_size, 8192),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1)
        )

    def forward(self, x):
        return self.model(x)

# Load trained model and evaluate on test set
model = DeepSynergyModel(input_size=X_test.shape[1])
model.load_state_dict(torch.load("deepsynergy_model.pt", map_location="cpu"))
model.eval()

with torch.no_grad():
    X_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_pred = model(X_tensor).squeeze().numpy()

mse_deepsynergy = mean_squared_error(y_test, y_pred)
print(f"DeepSynergy MSE on test set: {mse_deepsynergy:.2f}")


### Ergebnis (aus Terminallauf):

**DeepSynergy MSE (Testset Fold 0): 332.73**

Im Vergleich zu Random Forest (MSE: 277.02) liegt DeepSynergy deutlich höher.  
Trotzdem performt es besser als Ridge Regression (MSE: 425.96).  
Ein direkter Vergleich ist jedoch methodisch nicht ganz fair, da wie schon erwähnt, DeepSynergy nicht per Cross Validation evaluiert wurde.

### Zusammenfassung der Ergebnisse

| Modell           | Evaluationsmethode            | MSE     |
|------------------|-------------------------------|---------|
| Random Forest    | 5-fold Nested Cross Validation | 277.02  |
| Ridge Regression | 5-fold Nested Cross Validation | 425.96  |
| DeepSynergy      | Testsplit (Fold 0)             | 332.73  |


## Fazit

Obwohl DeepSynergy als tiefes neuronales Netz grundsätzlich in der Lage ist, komplexe Zusammenhänge zu modellieren, war die Umsetzung in diesem Projekt durch reduzierte Trainingszeit und vereinfachte Architektur limitiert.  
Ein vollständig trainiertes DeepSynergy mit optimierten Hyperparametern könnte bessere Ergebnisse liefern.

Die Ergebnisse bestätigen, dass klassische Modelle wie Random Forest auch in modernen Aufgabenstellungen konkurrenzfähig bleiben.
